In [1]:
import requests 
import numpy as np
import pandas as pd 
import sys
import xgboost 
import matplotlib.pyplot  as plt 
from numpy import interp
import matplotlib.pyplot as set_facecolor
from xgboost import XGBClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import tree #导入需要的模块
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GroupKFold, KFold,StratifiedShuffleSplit, GridSearchCV, cross_val_score,StratifiedKFold 
from sklearn.metrics import accuracy_score 
import matplotlib 
import scipy.stats as stats
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_auc_score,roc_curve,auc
from sklearn.model_selection import cross_val_score as CVS
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from IPython.display import display
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.metrics import confusion_matrix
from IPython.display import display, Image
import shap
from sklearn.metrics import classification_report
import webbrowser
from sklearn.metrics import precision_recall_curve,precision_score,recall_score,average_precision_score,ConfusionMatrixDisplay 
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE,RandomOverSampler
from collections import Counter
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# 读取CSV文件
data = pd.read_csv('lung1.csv')

# 查看数据的前几行
print(data.head())
from sklearn.model_selection import train_test_split
y= data.LungMetastasis #因子变量
X= data.drop('LungMetastasis',axis=1)#
X_train,Xtest,Y_train,Ytest = train_test_split(X,y,test_size=0.3,random_state=420)
traindata = pd.concat([X_train, Y_train], axis=1)
testdata = pd.concat([Xtest, Ytest], axis=1)
###traindata.to_csv('train_data.csv', index=False)
###testdata.to_csv('test_data.csv', index=False)####存为csv文件

   Age  Sex  Race  Histology  N stage  LungMetastasis  Tumor Size  \
0    3    1     2          0        1               0         1.8   
1    1    1     2          1        1               0         3.5   
2    3    0     2          2        1               1         8.5   
3    4    1     2          2        1               1         3.9   
4    2    1     2          1        0               0         7.5   

   Extrathyroidal extension  
0                         0  
1                         0  
2                         1  
3                         1  
4                         0  


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# 使用SMOTE进行过采样
smote = SMOTE(random_state=42,sampling_strategy=0.3)
Xtrain, Ytrain = smote.fit_resample(X_train, Y_train)

# 将过采样后的数据集合并
###traindata_resampled = pd.concat([Xtrain, Ytrain], axis=1)

# 导出过采样后的数据集
###traindata_resampled.to_csv('traindata_resampled.csv', index=False)

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 训练模型
model = XGBClassifier(
booster='gbtree',
objective='binary:logistic',eval_metric=['auc'],
max_depth=50,
n_estimators=100,#300
min_child_weight=5,
learning_rate=0.08,
random_state=0,
gamma=0.8,
reg_lambda=0.1,
reg_alpha=0.1,
scale_pos_weight=1,
subsample=0.5,
colsample_bytree=0.8,
seed=1,
n_jobs=-1)
model.fit(Xtrain, Ytrain)
# 保存模型
import joblib
joblib.dump(model, 'modelXGB.pkl')

['modelXGB.pkl']

In [14]:
%%writefile app.py
import streamlit as st
import numpy as np
import joblib

# 加载模型
model = joblib.load('modelXGB.pkl')

# Streamlit 用户界面
st.title("Lung metastasis risk probability Calculator")

# 用户输入
age = st.selectbox("Age(0: ＜25, 1: 25-39, 2: 40-54, 3: 55-69, 4: ≥70)", options=[0,1,2,3,4])

# 性别选择 
sex = st.selectbox("Sex (0: female, 1: male)", options=[0, 1])

# 种族选择 (0: 白人, 1: 黑人, 2: 亚裔, 3: 其他)
race = st.selectbox("Race (0: white, 1: black, 2: others)", options=[0, 1, 2])

# 组织学选择 (0: 腺癌, 1: 鳞状细胞癌, 2: 小细胞癌, 3: 其他)
histology = st.selectbox("Histology (0: MTC, 1: PTC, 2: ATC)", options=[0, 1, 2])

# N 期选择 (0: N0, 1: N1, 2: N2, 3: N3)
n_stage = st.selectbox("N stage (0: N0, 1: N1)", options=[0, 1])

# 甲状腺外延选择 (0: 否, 1: 是)
extrathyroidal_extension = st.selectbox("Extrathyroidal extension (0: no, 1: yes)", options=[0, 1])

# 肿瘤大小（连续变量）
tumor_size = st.number_input("Tumor Size (cm)", min_value=0.0, value=5.0)

# 按钮
if st.button("Calculate risk"):
    # 创建输入数据
    input_data = np.array([[age, sex, race, histology, 
                            n_stage, extrathyroidal_extension, tumor_size]])

    # 进行预测
    prediction = model.predict_proba(input_data)[:, 1]  # 获取转移的概率
    st.write(f"The risk probability of lung metastasis: {prediction[0]:.2%}")

Writing app.py


In [ ]:
###最后在命令输入符中运行streamlit run app.py